In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Set filepath for file to analyze
# NOTE: Edit this for each new file being run
zillow_csv_path = Path('Zillow_Data/Metro_median_sale_price_uc_sfrcondo_sm_month.csv')
zillow_metric_name = 'Rent Index'

#Set specific start year
start_year = 2018
end_year = 2023

In [3]:
# Create a data frame containing the data from the CSV
zillow_df = pd.read_csv(zillow_csv_path)
zillow_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,...,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31
0,102001,0,United States,country,NaN,176333.0,178833.0,181333.0,183000.0,182667.0,...,348167.0,343167.0,336667.0,330000.0,325833.0,324167.0,327500.0,335000.0,344167.0,353701.0
1,394913,1,"New York, NY",msa,NY,393517.0,391883.0,395217.0,401667.0,406667.0,...,578333.0,580333.0,573333.0,562500.0,559500.0,562314.0,561481.0,563814.0,569666.0,587000.0
2,753899,2,"Los Angeles, CA",msa,CA,460750.0,450750.0,443917.0,431500.0,418167.0,...,885667.0,885667.0,887333.0,890000.0,886667.0,878333.0,883667.0,899667.0,928000.0,946000.0
3,394463,3,"Chicago, IL",msa,IL,220265.0,223599.0,228265.0,232667.0,233833.0,...,310000.0,303333.0,295000.0,288333.0,283333.0,281667.0,281667.0,288874.0,300541.0,313874.0
4,394514,4,"Dallas, TX",msa,TX,142950.0,146950.0,150133.0,153167.0,153167.0,...,393939.0,387082.0,380415.0,376110.0,371967.0,369550.0,371583.0,379250.0,389833.0,397998.0


In [4]:
# Rename labels
zillow_df.rename(columns={'SizeRank': 'Size Rank', 'RegionName': 'Metro Area', 'StateName': 'State' }, inplace=True)
# Remove the RegionType column
zillow_df = zillow_df.drop('RegionType', axis = 1)
# Insert a new blank column to store the Size Segment
zillow_df['Size Segment'] = pd.NA
# Re-order columns so that Size Segment is next to Size Rank
columns = zillow_df.columns.tolist()
columns.remove('Size Segment')
columns.insert(2, 'Size Segment')
zillow_df = zillow_df[columns]
zillow_df.head()

,RegionID,Size Rank,Size Segment,Metro Area,State,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,...,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31
0,102001,0,<NA>,United States,NaN,176333.0,178833.0,181333.0,183000.0,182667.0,...,348167.0,343167.0,336667.0,330000.0,325833.0,324167.0,327500.0,335000.0,344167.0,353701.0
1,394913,1,<NA>,"New York, NY",NY,393517.0,391883.0,395217.0,401667.0,406667.0,...,578333.0,580333.0,573333.0,562500.0,559500.0,562314.0,561481.0,563814.0,569666.0,587000.0
2,753899,2,<NA>,"Los Angeles, CA",CA,460750.0,450750.0,443917.0,431500.0,418167.0,...,885667.0,885667.0,887333.0,890000.0,886667.0,878333.0,883667.0,899667.0,928000.0,946000.0
3,394463,3,<NA>,"Chicago, IL",IL,220265.0,223599.0,228265.0,232667.0,233833.0,...,310000.0,303333.0,295000.0,288333.0,283333.0,281667.0,281667.0,288874.0,300541.0,313874.0
4,394514,4,<NA>,"Dallas, TX",TX,142950.0,146950.0,150133.0,153167.0,153167.0,...,393939.0,387082.0,380415.0,376110.0,371967.0,369550.0,371583.0,379250.0,389833.0,397998.0


In [5]:
# Establish Size Segments as bins
size_segment_bins = [-1, 0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
size_segment_labels = ["National Average", "Top 100", "101 - 200", "201 - 300", "301 - 400", "401 - 500", "501 - 600", "601 - 700", "701 - 800", "801 - 900", "901 - 1000"]

In [6]:
#Filter data into specific range
def filter_data_by_date_range(df, start_date, end_date):
    filtered_df = df.loc[:, (df.columns >= start_date) & (df.columns <= end_date)]
    return filtered_df
# Specify the start and end dates for the time frame you want to analyze
start_date = '2018-01-31'
end_date = '2023-12-31'
# Filter the DataFrame based on the specified date range
filtered_zillow_df = filter_data_by_date_range(zillow_df, start_date, end_date)
# Display the filtered DataFrame
filtered_zillow_df.head()

,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31
0,216333.0,216667.0,219333.0,223667.0,228340.0,233246.0,235580.0,234906.0,230000.0,226650.0,...,314000.0,325000.0,335000.0,343333.0,348167.0,348167.0,343167.0,336667.0,330000.0,325833.0
1,371000.0,369333.0,373000.0,373417.0,381417.0,390083.0,401333.0,407754.0,403421.0,393421.0,...,510333.0,512250.0,525583.0,545250.0,561667.0,578333.0,580333.0,573333.0,562500.0,559500.0
2,593667.0,598667.0,608333.0,620000.0,627667.0,636000.0,640333.0,639333.0,631000.0,625250.0,...,822667.0,837333.0,849667.0,863000.0,875000.0,885667.0,885667.0,887333.0,890000.0,886667.0
3,208633.0,209800.0,215742.0,223775.0,231275.0,238000.0,238333.0,235000.0,225000.0,217500.0,...,264300.0,276000.0,289333.0,301667.0,308333.0,310000.0,303333.0,295000.0,288333.0,283333.0
4,239666.0,242166.0,245667.0,252540.0,258539.0,262839.0,265132.0,261633.0,255000.0,249333.0,...,362000.0,372156.0,380156.0,390156.0,395000.0,393939.0,387082.0,380415.0,376110.0,371967.0


In [9]:
# Insert segment values into Size Segment column
zillow_df['Size Segment'] = pd.cut(zillow_df["Size Rank"], bins=size_segment_bins, labels=size_segment_labels)
# Group by 'Size Segment' and calculate the mean for each segment
zillow_df_mean = zillow_df_filtered.groupby('Size Segment').mean()
# Reset the index for better display
zillow_df_mean.reset_index(inplace=True)
# Display the table
zillow_df_mean.head()

/var/folders/pp/tn2y0zwd30j8047f1n1vkprm0000gn/T/ipykernel_49637/3099292032.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  zillow_df_mean = zillow_df_filtered.groupby('Size Segment').mean()


,Size Segment,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,...,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31
0,National Average,216333.000000,216667.000000,219333.000000,223667.000000,228340.000000,233246.000000,235580.000000,234906.000000,230000.000000,...,314000.000000,325000.000000,335000.000000,343333.000000,348167.000000,348167.000000,343167.000000,336667.000000,330000.000000,325833.000000
1,Top 100,235601.622449,235954.193878,239040.234694,245378.244898,251599.061224,257450.173469,260142.642857,259845.806122,255314.693878,...,344950.878788,354734.111111,366758.787879,377131.070707,384277.714286,385791.367347,381622.050505,377449.838384,372860.848485,367932.787879
2,101 - 200,201035.714286,201551.479592,203518.153061,207140.816327,211316.030612,215368.469388,217672.214286,217548.193878,216342.377551,...,296824.030612,304365.224490,313204.510204,320670.561224,325951.224490,327454.795918,324815.153061,322233.479592,318885.795918,315726.020408
3,201 - 300,181788.842105,181502.063158,182841.747368,186027.873684,190801.378947,195699.463158,198077.600000,197848.800000,195752.400000,...,273245.347368,280643.831579,289291.884211,297306.852632,301705.747368,301264.936842,299962.821053,297037.305263,295796.189474,291030.347368
4,301 - 400,165733.565217,165484.880435,167383.076087,169119.880435,173079.434783,177049.282609,180485.315217,181803.206522,180714.847826,...,246332.000000,251554.260870,258882.402174,267069.369565,273296.086957,275502.086957,274869.413043,273152.445652,270280.771739,266132.141304
